***
### Metodos

Extraido do TFS os changesets:<br>
```python
tf hist /noprompt /recursive /format:detailed /version:D"1/1/2015"~D"31/12/2015" $/fPJ/PJ/build/release > 
```
Verificar as alterações:<br>
```python
"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" difference $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Cliente/uspjConfirmacaoPadrao.pas  /format:unix /noprompt /version:C555472~C555471 /ignorespace      
```
Pegar arquivo na versão:<br>
```python
"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" get $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas /version:C553471 /force /noprompt
```

** Utilizado arquivo TFS.ipynb para conversão em CSV e Changesets.ipynb.

In [23]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt

%matplotlib inline

#arquivos = ['release_changeset_20171.csv']
arquivos = ['C:\\temp\\main_changeset.csv']
TFS = '"D:\\\TFS\\\Microsoft Visual Studio 10.0\\\Common7\\\IDE\\\\tf.exe"'

In [24]:
dfChangesets = pd.read_csv(arquivos[0], encoding='ISO-8859-1', delimiter=';', header=None,
                           names= ["Salt", "Changeset", "Desenv", "Data", "Descricao", "File", "Filename"]) #,chunksize=20000, iterator=True)

In [35]:
#for chunk in dfChangesets:
#    print(chunk.shape)

caminhos = ['pj/Componentes', 'pj/Servidor', 'pj/Cliente'] #'pj/Classes'

for caminho in caminhos:
    print (caminho, dfChangesets[(dfChangesets.Filename.str.contains(caminho))].shape)

pj/Componentes (16887, 7)
pj/Servidor (11516, 7)
pj/Cliente (18703, 7)


In [28]:
dfChangesets.shape

(200775, 7)

In [ ]:
#for index, row in dfChangesets[(dfChangesets.Salt == '247275/4')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == '260601/1')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == '241736/1')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == 'Defect 120562')].iterrows():
#for index, row in dfChangesets[(dfChangesets.Salt == '1/1')].iterrows():

#chunk_iter = 0
#for chunk in dfChangesets:
#caminhos = ['spjConsultas']
for caminho in caminhos:
    geral = {'NOME':[],'SALT':[],'CHANGESET':[],'DESENV':[],'ARQUIVO':[]}
    for index, row in dfChangesets[(dfChangesets.Filename.str.contains(caminho))].iterrows():
    #for index, row in dfChangesets[(dfChangesets.Salt == '221124/1')].iterrows():
        if row['File'][-3:] != 'pas':
            continue
        metodos = []
        #print (row)
        diferenca = diff(row)
        #print (diferenca)
        linhas = line(diferenca)
        #print (linhas)
        arquivo = get(row)
        #print (arquivo[-3:])
        if os.path.isfile(arquivo):
            if arquivo[-3:] == 'pas':
                for linha in linhas:
                    nome = metodo(arquivo, int(linha))
                    if nome != '':
                        if not ([nome, arquivo] in metodos):
                            metodos.append([nome, arquivo])
                            #print (nome)
        for nome, arquivo in metodos:
            #print (nome)
            geral['NOME'].append(nome)
            geral['SALT'].append(row["Salt"])
            geral['CHANGESET'].append(row["Changeset"])
            geral['DESENV'].append(row["Desenv"])
            geral['ARQUIVO'].append(arquivo)

    data_frame = pd.DataFrame.from_dict(geral, dtype=object)
    data_frame.drop_duplicates()
    data_frame.to_csv(path_or_buf=str(caminho).replace('pj/', '') +'.csv', index=False)
    #data_frame.to_csv(path_or_buf='nome'+str(chunk_iter)+'.csv', index=False)
    #chunk_iter += 1
    print (caminho, data_frame.shape)

In [72]:
#dfChangesets[(dfChangesets.Salt == '241736/1')]
#dfChangesets[(dfChangesets.Filename.str.contains('/fPJ/PJ/build/main/pjDev/src/pj/Servidor'))]
dfChangesets[(dfChangesets.Filename.str.contains('spjComponentesDT'))]

,Salt,Changeset,Desenv,Data,Descricao,File,Filename
1038,251910/1,551807,robledo.fortuna,13/12/2017,Apresentar mensagem de erro no recebimento de ...,uspjConjuntoDeDadosRegistro.pas,/fPJ/PJ/build/main/pjDev/src/pj/Componentes/sp...
34182,170146/1,550418,olimar.santos,1/12/2017,Migração Tokyo,uspjComponentesRegistro.pas,/fPJ/PJ/build/main/pjDev/src/pj/Componentes/sp...
34183,170146/1,550418,olimar.santos,1/12/2017,Migração Tokyo,uspjConjuntoDeDadosRegistro.pas,/fPJ/PJ/build/main/pjDev/src/pj/Componentes/sp...
56797,174268/1/2,550091,henrique.ramos,30/11/2017,"Remoção solicitada por Ulder.Junior, branch cr...",spjComponentesDT;X66459,/fPJ/PJ/build/main/features/121705/pjDev/src/p...
57673,174268/1/2,550091,henrique.ramos,30/11/2017,"Remoção solicitada por Ulder.Junior, branch cr...",uspjComponentesRegistro.pas;X66459,/fPJ/PJ/build/main/features/121705/pjDev/src/p...
57674,174268/1/2,550091,henrique.ramos,30/11/2017,"Remoção solicitada por Ulder.Junior, branch cr...",uspjConjuntoDeDadosRegistro.pas;X66459,/fPJ/PJ/build/main/features/121705/pjDev/src/p...
82308,255191/1,550076,ulder.junior,30/11/2017,Branched from $/fPJ/PJ/build/main/pjDev,spjComponentesDT,/fPJ/PJ/build/main/features/121705/pjDev/src/p...
83184,255191/1,550076,ulder.junior,30/11/2017,Branched from $/fPJ/PJ/build/main/pjDev,uspjComponentesRegistro.pas,/fPJ/PJ/build/main/features/121705/pjDev/src/p...
83185,255191/1,550076,ulder.junior,30/11/2017,Branched from $/fPJ/PJ/build/main/pjDev,uspjConjuntoDeDadosRegistro.pas,/fPJ/PJ/build/main/features/121705/pjDev/src/p...
86768,248798/29,550067,rafael.wagner,30/11/2017,Merge feature WIL_2017_4-SP2,uspjComponentesRegistro.pas,/fPJ/PJ/build/main/pjDev/src/pj/Componentes/sp...


In [ ]:
df = pd.DataFrame(geral)
df.to_csv('C:\\temp\\geral.csv', index=False, header=False, encoding ='ISO-8859-1', sep=';')

In [4]:
#File: D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Cliente\uspjConfirmacaoPadrao.pas
#===================================================================
#150,151c150
#<   // 26/12/2017 - Wilian.Ivo - Salt: 262749/1
#<   if not (nErroLogin in [nLoginOK, nLoginSenhaPrestesAExpirar]) then
#---
#>   if nErroLogin > 0 then
#===================================================================

def diff(row):
    comando = TFS + ' difference ' + '$' + row['Filename'] + ' /format:unix /noprompt '
    comando += '/version:' + 'C' + str(row['Changeset']) + '~' + 'C' + str(int(str(row['Changeset']).replace('C', '')) - 1) + ' /ignorespace'
    #print (comando)
    pipe = os.popen(comando)
    retorno = pipe.read()
    pipe.close()
    #print (a)
    return retorno

teste = {'Filename':'/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Cliente/uspjConfirmacaoPadrao.pas', 'Changeset':'555472'}
print (diff(teste))

teste = {'Filename':'/fPJ/PJ/build/release/4.0.18/USP/pjDev/src/3rd_party/edtDev/src/Cliente/uedtEditor5.pas', 'Changeset':'551248'}
print (diff(teste))


File: D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Cliente\uspjConfirmacaoPadrao.pas
150,151c150
<   // 26/12/2017 - Wilian.Ivo - Salt: 262749/1
<   if not (nErroLogin in [nLoginOK, nLoginSenhaPrestesAExpirar]) then
---
>   if nErroLogin > 0 then

File: D:\UNJ\fPJ\PJ\build\release\4.0.18\USP\pjDev\src\3rd_party\edtDev\src\Cliente\uedtEditor5.pas
1411,1412d1410
<     // 07/12/2017 - Anderson.Albuquerque - SALT 260601/1
<     procedure PassarParametrosParaEnvioDeEmail(oForm: TspForm); virtual;
15266,15267c15264
<     // 07/12/2017 - Anderson.Albuquerque - SALT 260601/1
<     PassarParametrosParaEnvioDeEmail(oForm);
---
>     oForm.PassaParametro(sPRM_ADD_ARQS_EMAIL, FPathArquivoEmail);
21121,21126d21117
< // 07/12/2017 - Anderson.Albuquerque - SALT 260601/1
< procedure TfedtEditor5.PassarParametrosParaEnvioDeEmail(oForm: TspForm);
< begin
<   oForm.PassaParametro(sPRM_ADD_ARQS_EMAIL, FPathArquivoEmail);
< end;
< 



In [2]:
pipe = os.popen('"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" difference $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas /format:unix /noprompt /version:C553471~C553470 /ignorespace')
a = pipe.read()
pipe.close()
print (a)


File: D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Servidor\uspjSelectFluxoObjetoBasico.pas
544a545
>     wflSelectPrincipal.AddColuna('p.flLitispendencia as cc_flLitispendencia');
546d546
<     //13/12/2017 - Wilian.Ivo - Salt: 255004/2
548,559c548,552
<       FSQL.Decode(FSQL.Value('pSup.flLitispendencia', 'p.flLitispendencia'),
<       [QuotedStr(cLitisPendente), QuotedStr(sLITIS_PENDENTE), QuotedStr(cLitisSuspeita),
<       QuotedStr(sLITIS_SUSPEITA), QuotedStr(cLitisConfirmada), QuotedStr(sLITIS_CONFIRMADA),
<       QuotedStr(cLitisConexao), QuotedStr(sLITIS_CONEXAO), QuotedStr(cLitisDescartada),
<       QuotedStr(sLITIS_DESCARTADA), QuotedStr(sLITIS_INEXISTENTE)]) + ' as cc_deLitispendencia');
< 
<     //26/12/2017 - Wilian.Ivo - Salt: 255004/2
<     if (oParametrosConsultaObjetoFila.TestarCampoNaProjecao('cc_deLitispendencia')) then
<       wflSelectPrincipal.AddColuna(FSQL.Value('pSup.flLitispendencia', 'p.flLitispendencia') +
<         ' as cc_flLitispendencia')
<   

In [5]:
#D:\UNJ\fPJ\PJ\build\release\4.20\producao\pjDev\src\pj\Servidor:
#Replacing uspjSelectFluxoObjetoBasico.pas

def get(row):
    #pipe = os.popen('"D:\\TFS\\Microsoft Visual Studio 10.0\\Common7\\IDE\\tf.exe" get $/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas /version:C553471 /force /noprompt')
    comando = TFS + ' get $' + row['Filename'] + ' /version:' + 'C'
    comando += str(row['Changeset']) + ' /force /noprompt'
    #print(comando)
    pipe = os.popen(comando)
    retorno = pipe.read()
    pipe.close()
    #print (retorno)
    linhas = retorno.splitlines()
    #print(len(linhas))
    caminho = ''
    arquivo = ''
    if linhas[0] != 'All files are up to date.':
        try:
            caminho = linhas[0][:len(linhas[0])-1]
            #caminho = caminho.replace('\\', '\\\\')
            arquivo = linhas[1].replace('Replacing ', '').replace('Getting ', '')
        except:
            print ('linhas:',linhas)
    return caminho + '\\' + arquivo

teste = {'Filename':'/fPJ/PJ/build/release/4.20/producao/pjDev/src/pj/Servidor/uspjSelectFluxoObjetoBasico.pas', 'Changeset':'553471'}
get(teste)

'D:\\UNJ\\fPJ\\PJ\\build\\release\\4.20\\producao\\pjDev\\src\\pj\\Servidor\\uspjSelectFluxoObjetoBasico.pas'

In [6]:
def line(diff):
    retorno = []
    for linha in diff.splitlines():
        if linha[0].isdigit():
            registro = linha.split(',')[0]
            if registro.isdigit():
                retorno.append(registro)
            else:
                registro = linha.split('c')[0]
                if registro.isdigit():
                    retorno.append(registro)
                else:
                    registro = linha.split('d')[0]
                    if registro.isdigit():
                        retorno.append(registro)
    return retorno


In [13]:
def metodo(arquivo, linha):
    #f = open('D:\\UNJ\\fPJ\\PJ\\build\\release\\4.20\\producao\\pjDev\\src\\pj\\Servidor\\uspjSelectFluxoObjetoBasico.pas')
    f = open(arquivo)
    lines = f.readlines()
    if linha > len(lines):
        print ('arquivo:', arquivo)
        metodo = ''
    else:
        while (linha > 0):
            texto = ''
            try:
                texto = lines[linha]
            except:
                #print ('lines:', lines)
                texto = ''
                metodo = ''
            if re.match('function.*\..*\;', texto) \
            or re.match('procedure.*\..*\;', texto) \
            or re.match('function.*\..*\(', texto) \
            or re.match('procedure.*\..*\(', texto): 
                metodo = texto
                break
            linha -= 1
        else:
            metodo = ''
    return metodo.replace('\n', '')

metodo('D:\\UNJ\\fPJ\\PJ\\build\\release\\4.20\\producao\\pjDev\\src\\pj\\Servidor\\uspjSelectFluxoObjetoBasico.pas', 800)

'procedure TspjSelectFluxoObjetoBasico.AdicionarCondicoes;'

In [21]:
#Salt Changeset Desenv Data Descricao File Filename
row = {'Filename':'/fPJ/PJ/build/main/pjDev/src/3rd_party/wflDev/src/Classes/uwflVisualizaFluxoTrabalho.pas',
       'Changeset':'548806'}

print ('\nDIFF ------------------------')
diferenca = diff(row)
print (diferenca)

print ('\nLINH ------------------------')
linhas = line(diferenca)
print (linhas)

print ('\nFILE ------------------------')
arquivo = get(row)
print (arquivo)

for linha in linhas:
    print ('\nNOME ------------------------')
    nome = metodo(arquivo, int(linha))
    print (nome)



DIFF ------------------------
File: D:\UNJ\fPJ\PJ\build\main\pjDev\src\3rd_party\wflDev\src\Classes\uwflVisualizaFluxoTrabalho.pas
6324,6326d6323
< var
<   // 14/10/2010 - RSF - SALT 69191/1
<   bMudouFluxoTrabalho: boolean;


LINH ------------------------
['6324']

FILE ------------------------
D:\UNJ\fPJ\PJ\build\main\pjDev\src\3rd_party\wflDev\src\Classes\uwflVisualizaFluxoTrabalho.pas

NOME ------------------------
procedure TfwflVisualizaFluxoTrabalho.grDadosChangeNode(Sender: TObject;
